# Generate density split cross-correlation functions

In [1]:
import os
import tempfile
import time
import copy

import numpy as np
import scipy
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (10, 8)
plt.rcParams.update({'font.size': 12})

from pypower import CatalogMesh, MeshFFTPower, CatalogFFTPower, PowerSpectrumStatistics, utils, setup_logging
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, NaturalTwoPointEstimator, project_to_multipoles, BoxSubsampler, project_to_wp, utils, setup_logging
from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog
from cosmoprimo import *

import densitysplit 
from densitysplit import catalog_data, density_split
from bin.density_split_mocks_functions import split_density, compute_densitySplit_CCF
from densitysplit.pk_model import *
from densitysplit.utils import *
from densitysplit.corr_func import *

## Data and output directories

In [2]:
data_dir = '/feynman/work/dphp/mp270220/data/'
output_dir = '/feynman/work/dphp/mp270220/outputs/'

## Get data

In [3]:
#catalog_name = 'AbacusSummit_1Gpc_z1.175'
catalog_name = 'AbacusSummit_2Gpc_z1.175'
#catalog_name = 'AbacusSummit_2Gpc_z0.800'
#catalog_name = 'mock'

catalog = catalog_data.Data.load(data_dir+catalog_name+'.npy')
catalog.shift_boxcenter(-catalog.offset)

## Apply RSD

In [4]:
# Abacus summit base cosmology
cosmo_abacus = fiducial.AbacusSummitBase()
bg = cosmo_abacus.get_background()
hz = bg.hubble_function(catalog.redshift)

catalog.set_rsd(hz=hz)

## Split density

In [5]:
# Density mesh
cellsize = 10
resampler = 'tsc'
nsplits = 3

catalog_density = split_density(catalog, cellsize, resampler, nsplits, use_rsd=True, save=False)

[000001.21] [0/1] 05-20 10:29  CatalogMesh               INFO     Slab 0 ~ 4194304 / 23463294.
[000002.22] [0/1] 05-20 10:29  CatalogMesh               INFO     Painted 4194304 out of 23463294 objects to mesh.
[000002.22] [0/1] 05-20 10:29  CatalogMesh               INFO     Slab 4194304 ~ 8388608 / 23463294.
[000003.23] [0/1] 05-20 10:29  CatalogMesh               INFO     Painted 8388608 out of 23463294 objects to mesh.
[000003.23] [0/1] 05-20 10:29  CatalogMesh               INFO     Slab 8388608 ~ 12582912 / 23463294.
[000004.23] [0/1] 05-20 10:29  CatalogMesh               INFO     Painted 12582912 out of 23463294 objects to mesh.
[000004.23] [0/1] 05-20 10:29  CatalogMesh               INFO     Slab 12582912 ~ 16777216 / 23463294.
[000005.23] [0/1] 05-20 10:29  CatalogMesh               INFO     Painted 16777216 out of 23463294 objects to mesh.
[000005.24] [0/1] 05-20 10:29  CatalogMesh               INFO     Slab 16777216 ~ 20971520 / 23463294.
[000006.24] [0/1] 05-20 10:29  Cat

## Correlation galaxies-galaxies

### Correlation function in each density split

In [6]:
edges = (np.linspace(0., 150., 51), np.linspace(-1, 1, 201))
los='x'

In [7]:
results = compute_densitySplit_CCF(catalog_density, edges, los, use_rsd=True, save=True, output_dir=output_dir, name=catalog_name, randoms_size=4)

rsd positions found
[000038.40] [0/1] 05-20 10:30  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000038.40] [0/1] 05-20 10:30  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[000070.10] [0/1] 05-20 10:30  TwoPointCorrelationFunction INFO     Analytically computing two-point counts R1R2.
[000070.15] [0/1] 05-20 10:30  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000070.15] [0/1] 05-20 10:30  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[000155.45] [0/1] 05-20 10:32  TwoPointCorrelationFunction INFO     Analytically computing two-point counts R1R2.
[000155.46] [0/1] 05-20 10:32  TwoPointCorrelationFunction INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000155.46] [0/1] 05-20 10:32  TwoPointCorrelationFunction INFO     Computing two-point counts D1D2.
[001277.98]